In [37]:
#import sys
#conda install --yes --prefix {sys.prefix} tksheet
#!pip install tksheet       

In [38]:
import pandas as pd
import sys
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import ( FigureCanvasTkAgg, NavigationToolbar2Tk)
from matplotlib.figure import Figure
import tkinter as tk
from tkinter import ttk
import tksheet

LARGE_FONT= ("Verdana", 12)

data_sec = pd.read_csv('eve1.csv')
data_sec.head()

,Unnamed: 0,timestamp,flow_id,pcap_cnt,event_type,src_ip,src_port,dest_ip,dest_port,proto,...,stats.app_layer.tx.dcerpc_udp,stats.app_layer.tx.dns_udp,stats.app_layer.tx.nfs_udp,stats.app_layer.tx.krb5_udp,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,0,2014-08-01T02:52:29.986487+0200,5.406114e+14,1.0,dns,172.16.165.132,57372.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-08-01T02:52:30.181417+0200,5.406114e+14,2.0,dns,172.16.165.132,57372.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2014-08-01T02:52:30.567661+0200,1.955962e+15,37.0,dns,172.16.165.132,51240.0,172.16.165.2,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2014-08-01T02:52:30.664068+0200,6.416762e+14,42.0,http,172.16.165.132,49476.0,95.85.17.107,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2014-08-01T02:52:30.664068+0200,6.416762e+14,42.0,fileinfo,95.85.17.107,80.0,172.16.165.132,49476.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Które kolumny nas interesują: timestamp, 
Alert.action, alert.category, alert.signature, alert.severity, alert.metadata.created_at, alert.metadata.updated_at

+ url i hostname: http.hostname, http.url,


 count_alerts - własna miara

pcap_cnt, event_type, src_ip, src_port, dest_ip, dest_port, proto, http.hostname, http.url, http.http_user_agent

alert.metadata.created_at is not null!!!!

Ewentualnie: flow_id, dns.type, dns.id, dns.rrname, dns.rrtype, dns.version, dns.answers, htttp.http_content_type, http.http_method, http.protocol, http.status, fileinfo.sid

In [39]:
data_alerts_only = data_sec.loc[data_sec['alert.metadata.created_at'].notnull(), ['alert.metadata.created_at','alert.category', 'alert.severity', 'http.hostname', 'http.url' ]]
#print(data_alerts_only)
print(type(data_alerts_only))

<class 'pandas.core.frame.DataFrame'>


In [40]:
grouped_by_category = data_alerts_only.groupby('alert.category').count()
data_category = grouped_by_category[[ "alert.metadata.created_at"]]

grouped_by_severity = data_alerts_only.groupby('alert.severity').count()
data_severity = grouped_by_severity[[ "alert.metadata.created_at"]]

#print(data_smaller)

In [41]:
categories = data_category.index.tolist()
categories_counts = data_category['alert.metadata.created_at'].tolist()

severities = data_severity.index.tolist()
severities_counts = data_severity['alert.metadata.created_at'].tolist()

#print(severities)
#print(categories_counts)

In [ ]:

class SecurityApp(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        tk.Tk.wm_title(self, "Security App")
        
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand = True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (StartPage, PagePieChart, PagePieChart2, PageSummary):  #tu trzeba dodac każde nowe okienko
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        self.show_frame(StartPage)

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

        
class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self,parent)
        label = tk.Label(self, text="Start Page", font=LARGE_FONT)
        label.pack(pady=10,padx=10)
        
        button3 = ttk.Button(self, text="See Alerts per Category", command=lambda: controller.show_frame(PagePieChart))
        button3.pack()
        
        button4 = ttk.Button(self, text="See Alerts per Severity", command=lambda: controller.show_frame(PagePieChart2))
        button4.pack()
        
        button5 = ttk.Button(self, text="See Alerts Summary", command=lambda: controller.show_frame(PageSummary))
        button5.pack()
        


class PagePieChart(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Alerts per Category", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        button2 = ttk.Button(self, text="See Alerts per Severity",command=lambda: controller.show_frame(PagePieChart2))
        button2.pack()
        
        button3 = ttk.Button(self, text="See Alerts Summary",command=lambda: controller.show_frame(PageSummary))
        button3.pack()

        labels = categories
        sizes = categories_counts

        f, a = plt.subplots()
        a.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=False, startangle=90)
        a.axis('equal')  

        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw() 
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)  #na dole wykresu
        toolbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        
class PagePieChart2(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Alerts per Severity", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        button2 = ttk.Button(self, text="See Alerts per Category",command=lambda: controller.show_frame(PagePieChart))
        button2.pack()
        
        button3 = ttk.Button(self, text="See Alerts Summary",command=lambda: controller.show_frame(PageSummary))
        button3.pack()

        labels = severities
        sizes = severities_counts

        f, a = plt.subplots()
        a.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=False, startangle=90)
        a.axis('equal')  

        canvas = FigureCanvasTkAgg(f, self)
        canvas.draw() 
        canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

        toolbar = NavigationToolbar2Tk(canvas, self)  #na dole wykresu
        toolbar.update()
        canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        
        
class PageSummary(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        label = tk.Label(self, text="Summary of Alerts", font=LARGE_FONT)
        label.pack(pady=10,padx=10)

        button1 = ttk.Button(self, text="Back to Home", command=lambda: controller.show_frame(StartPage))
        button1.pack()
        
        
        
        ## Treeview Widget
        tv1 = ttk.Treeview(self)

        treescrolly = tk.Scrollbar(self, orient="vertical", command=tv1.yview) # command means update the yaxis view of the widget
        treescrollx = tk.Scrollbar(self, orient="horizontal", command=tv1.xview) # command means update the xaxis view of the widget
        tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
        treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
        treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget
        
        
        df = data_alerts_only
        
        tv1.delete(*tv1.get_children())
        tv1["column"] = list(df.columns)
        tv1["show"] = "headings"
        
        for column in tv1["columns"]:
            tv1.heading(column, text=column) # let the column heading = column name

        df_rows = df.to_numpy().tolist() # turns the dataframe into a list of lists
        for row in df_rows:
            tv1.insert("", "end", values=row) 
            
            
        #https://gist.github.com/RamonWill/0686bd8c793e2e755761a8f20a42c762
        


        
        """
        sheet = tksheet.Sheet(self)   #nie działa dla dataframe tylko dla jakiejs własnej tabeli / list
        sheet.pack()
        sheet.set_sheet_data(data = data_alerts_only)
        
        
        #sheet.set_sheet_data([[f"{ri+cj}" for cj in range(4)] for ri in range(1)])
       
        #dodatkowe funkcje tabeli
        sheet.enable_bindings(("single_select","row_select","column_width_resize","arrowkeys","right_click_popup_menu",
        "rc_select","rc_insert_row","rc_delete_row","copy","cut","paste","delete","undo","edit_cell"))
        """
        
app = SecurityApp()
app.mainloop()
        
        
